<a href="https://colab.research.google.com/github/talhaanwarch/my_pytorch/blob/master/advance/3D_segmentation_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [2]:
%%capture
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install "monai-weekly[nibabel]"
!pip install -U pytorch-lightning
!pip install torchmetrics
!pip install -U tqdm
%matplotlib inline

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1606217d-5e2f-b5ce-dd69-b1bde23b6d37)


In [4]:
import tqdm
print(tqdm.__version__)
assert tqdm.__version__ >='4.47.0', 'tqdm version >=4.47.0'

4.61.2


In [5]:
#download data
import os
if os.path.isfile('Task09_Spleen.tar') is False:
  !wget https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar

In [6]:
import tarfile
if os.path.isdir('Task09_Spleen') is False:
  my_tar = tarfile.open('/content/Task09_Spleen.tar')
  my_tar.extractall('/content/')
  my_tar.close()

In [1]:
from glob import glob
import os
images = sorted(glob("Task09_Spleen/imagesTr/*.nii.gz"))
segs = sorted(glob("Task09_Spleen/labelsTr/*.nii.gz"))
data_dicts = [
            {"image": image_name, "label": label_name}
            for image_name, label_name in zip(images, segs)
        ]
train_files, val_files = data_dicts[:-9], data_dicts[-9:]

In [2]:
import nibabel as nib
for i in range(len(train_files)):
  img = nib.load(train_files[i]['image']).get_fdata()
  lab = nib.load(train_files[i]['label']).get_fdata()
  print(img.shape,lab.shape)
  break

(512, 512, 55) (512, 512, 55)


In [3]:
import nibabel as nib
raw_val_data=[]
for i in range(len(val_files)):
  img = nib.load(val_files[i]['image']).get_fdata()
  lab = nib.load(val_files[i]['label']).get_fdata()
  print(img.shape,lab.shape)
  raw_val_data.append((img,lab))
  

(512, 512, 33) (512, 512, 33)
(512, 512, 50) (512, 512, 50)
(512, 512, 135) (512, 512, 135)
(512, 512, 97) (512, 512, 97)
(512, 512, 101) (512, 512, 101)
(512, 512, 80) (512, 512, 80)
(512, 512, 60) (512, 512, 60)
(512, 512, 31) (512, 512, 31)
(512, 512, 41) (512, 512, 41)


In [4]:
from  monai import transforms as T
train_transforms = T.Compose(
            [
        T.LoadImaged(keys=["image", "label"]),
        T.AddChanneld(keys=["image", "label"]),
        T.Spacingd(keys=["image", "label"],pixdim=(1.5, 1.5, 2.0),mode=("bilinear", "nearest")),
        T.ScaleIntensityd(keys=["image"]),
        T.CropForegroundd(keys=["image", "label"], source_key="image"),
        T.RandSpatialCropSamplesd(
            keys=["image", "label"],
            roi_size =(64,64,64),
            num_samples=10, 
            random_size =False,
        ),
        T.ToTensord(keys=["image", "label"]),
            ]
        )

# val_transforms = T.Compose(
#             [
#         T.LoadImaged(keys=["image", "label"]),
#         T.AddChanneld(keys=["image", "label"]),
#         T.Spacingd(keys=["image", "label"],pixdim=(1.5, 1.5, 2.0),mode=("bilinear", "nearest")),
#         T.ScaleIntensityd(keys=["image"]),
#         T.CropForegroundd(keys=["image", "label"], source_key="image"),
#         T.ToTensord(keys=["image", "label"]),
#             ]
#         )

# test_transforms = T.Compose(
#             [
#         T.LoadImaged(keys=["image"]),
#         T.AddChanneld(keys=["image"]),
#         T.Spacingd(keys=["image"],pixdim=(1.5, 1.5, 2.0),mode=("bilinear", "nearest")),
#         T.ScaleIntensityd(keys=["image"]),
#         T.CropForegroundd(keys=["image"], source_key="image"),
#         T.RandCropByPosNegLabeld(
#             keys=["image"],
#             spatial_size=(64, 64, 64),
#             pos=1,
#             neg=1,
#             num_samples=8,
#             image_key="image",
#             image_threshold=0,
#         ),
#         T.ToTensord(keys=["image"]),
#             ]
#         )


post_trans = T.Compose(
    [T.ToTensor(), T.Activations(sigmoid=True), T.AsDiscrete(threshold_values=True)]
)

In [5]:
from pytorch_lightning import seed_everything, LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint
from monai.losses import DiceLoss
from torch.utils.data import DataLoader, Dataset
from monai.data import CacheDataset, list_data_collate
from pytorch_lightning.loggers import TensorBoardLogger
from monai.networks.nets import UNet
import torch.nn as nn
import torch
import torchmetrics


In [6]:
from monai.networks.layers import Norm
from monai.inferers import sliding_window_inference

class OurModel(LightningModule):
  def __init__(self):
    super(OurModel,self).__init__()
    #architecute
    self.layer = UNet(
                      dimensions=3,
                      in_channels=1,
                      out_channels=2,
                      channels=(16, 32, 64, 128, 256),
                      strides=(2, 2, 2, 2),
                      num_res_units=2,
                      norm=Norm.BATCH,
                  )

  #parameters
    self.lr=1e-4
    self.bs=16
    self.numworker=2
    self.criterion=DiceLoss(to_onehot_y=True, softmax=True)
    self.metrics = torchmetrics.IoU(num_classes=2)
  def forward(self,x):
    return self.layer(x)


  def configure_optimizers(self):
    opt=torch.optim.Adam(self.parameters(), lr=self.lr)
    return opt

  def train_dataloader(self):
    ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0, num_workers=self.numworker)
    loader=DataLoader(ds, batch_size=self.bs, shuffle=True,num_workers=self.numworker, collate_fn=list_data_collate)
    return loader

  def training_step(self,batch,batch_idx):
    image,segment=batch["image"], batch["label"]
    out=self(image)
    loss=self.criterion(out,segment)

    self.log('train/loss', loss, on_epoch=True,prog_bar=True)
    return loss

  def val_dataloader(self):
    ds = CacheDataset(data=val_files, transform=train_transforms,cache_rate=1.0, num_workers=self.numworker)
    loader=DataLoader(ds, batch_size=self.bs, shuffle=False,num_workers=self.numworker, collate_fn=list_data_collate)
    return loader
    
  def validation_step(self,batch,batch_idx):
    image,segment=batch["image"], batch["label"]
    roi_size = (32, 32, 32)
    sw_batch_size = 2
    outputs = sliding_window_inference(image, roi_size, sw_batch_size, self.forward)
    loss=self.criterion(outputs,segment)
    iou=self.metrics(outputs.type(torch.float),segment.type(torch.long))
    self.log('val/loss', loss, on_epoch=True,prog_bar=True)
    self.log('val/iou', iou, on_epoch=True,prog_bar=True)
    return loss


In [10]:
ls drive/MyDrive/

'86_ Talha Anwar.mp4'   data/            spleen200-v1.ckpt   spleen_state600.pt
 backup/                mam/             spleen200-v2.ckpt   ultrasound.onnx
'Colab Notebooks'/      spleen200.ckpt   spleen500.ckpt


In [11]:
model = OurModel()
logger = TensorBoardLogger("logs", name="my_logs")
checkpoint_callback = ModelCheckpoint(monitor='val/loss',dirpath='/content/drive/MyDrive/',
                                      filename='spleen500' )
trainer = Trainer(max_epochs=500, auto_lr_find=False, auto_scale_batch_size=False,
                  #tpu_cores=8,precision=16,
                  gpus=-1,
                  logger=logger,
                  progress_bar_refresh_rate=30,
                  resume_from_checkpoint='/content/drive/MyDrive/spleen500.ckpt',
                  callbacks=[checkpoint_callback]
                  )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [12]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type     | Params
---------------------------------------
0 | layer     | UNet     | 4.8 M 
1 | criterion | DiceLoss | 0     
2 | metrics   | IoU      | 0     
---------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.236    Total estimated model params size (MB)
Restored states from the checkpoint file at /content/drive/MyDrive/spleen500.ckpt


Validation sanity check: 0it [00:00, ?it/s]


Loading dataset: 100%|██████████| 32/32 [01:34<00:00,  2.96s/it]


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:897: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn('Detected KeyboardInterrupt, attempting graceful shutdown...')


In [13]:
torch.save(model.state_dict(), '/content/drive/MyDrive/spleen_state500.pt')

In [14]:
#model.load_state_dict(torch.load('/content/drive/MyDrive/spleen_state500.pt'))


# inference

In [15]:
post_transforms = T.Compose([
        T.ToTensord(keys="pred"), 
        T.Activationsd(keys="pred", sigmoid=True),
        T.AsDiscreted(keys="pred", threshold_values=True),
        T.Invertd(
            keys=["pred",'label'],  # invert the `pred` data field, also support multiple fields
            transform=train_transforms,
            orig_keys="image",  # get the previously applied pre_transforms information on the `img` data field,
                              # then invert `pred` based on this information. we can use same info
                              # for multiple fields, also support different orig_keys for different fields
            meta_keys=["pred_meta_dict","label_meta_dict"],  # key field to save inverted meta data, every item maps to `keys`
            orig_meta_keys="image_meta_dict",  # get the meta data from `img_meta_dict` field when inverting,
                                             # for example, may need the `affine` to invert `Spacingd` transform,
                                             # multiple fields can use the same meta data to invert
            meta_key_postfix="meta_dict",  # if `meta_keys=None`, use "{keys}_{meta_key_postfix}" as the meta key,
                                           # if `orig_meta_keys=None`, use "{orig_keys}_{meta_key_postfix}",
                                           # otherwise, no need this arg during inverting
            nearest_interp=True,  # change to use "nearest" mode in interpolation when inverting
            to_tensor=True,  # convert to PyTorch Tensor after inverting
        ),
        #SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir="./out", output_postfix="seg", resample=False),
    ])

In [ ]:
val_dataloader=model.val_dataloader()



Loading dataset:   0%|          | 0/9 [00:00<?, ?it/s]

Loading dataset:  11%|█         | 1/9 [00:02<00:21,  2.65s/it]

Loading dataset:  22%|██▏       | 2/9 [00:05<00:17,  2.55s/it]

Loading dataset:  33%|███▎      | 3/9 [00:10<00:22,  3.67s/it]

Loading dataset:  44%|████▍     | 4/9 [00:13<00:18,  3.62s/it]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
model.eval()
device = torch.device("cuda:0")
model.to(device)
ch=32
with torch.no_grad():
    for i, val_data in enumerate(val_dataloader):
        roi_size = (32, 32, 32)
        sw_batch_size = 2
        output = sliding_window_inference(
            val_data["image"].to(device), roi_size, sw_batch_size, model
        )
        # plot the slice [:, :, 80]
        
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(val_data["image"][0, 0, :, :, ch], cmap="gray")
        plt.subplot(1, 3, 2)
        plt.title(f"label {i}")
        plt.imshow(val_data["label"][0, 0, :, :, ch])
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(torch.argmax(
            output, dim=1).detach().cpu()[0, :, :, ch])
        plt.show()
        break
        

In [ ]:
from monai.data import decollate_batch
model.eval()
post_data=[]
with torch.no_grad():
  for d in val_dataloader:
    images = d["image"].to(device)
    # define sliding window size and batch size for windows inference
    infer_outputs = sliding_window_inference(inputs=images, roi_size=(32 , 32, 32), sw_batch_size=2, predictor=model)
    infer_outputs = decollate_batch(infer_outputs)
    for (infer_output, infer_output_data) in zip(infer_outputs, decollate_batch(d)):
      infer_output_data["pred"] = infer_output
      post_data.append(post_transforms(infer_output_data))
      break
    break
    

In [ ]:
out=post_data[0]
out['image'].shape,out['label'].shape,out['pred'].shape

In [ ]:
raw_val=raw_val_data[0]
raw_val[0].shape,raw_val[1].shape#raw label and segment

In [ ]:
fig,ax=plt.subplots(33,3,figsize=(10,60))

for i in range(33):
  ax[i,0].imshow(raw_val[1][:,:,i],cmap='gray')#raw image
  ax[i,1].imshow(out['label'][0,:,:,i],cmap='gray')#inverted image
  ax[i,2].imshow(torch.argmax(out['pred'],0)[:,:,i],cmap='gray')#predicted image
plt.subplots_adjust(wspace=0, hspace=-.5)


# test

In [ ]:
post_transforms = T.Compose([
        T.ToTensord(keys="pred"), 
        T.Activationsd(keys="pred", sigmoid=True),
        T.AsDiscreted(keys="pred", threshold_values=True),
        T.Invertd(
            keys=["pred",'label'],  # invert the `pred` data field, also support multiple fields
            transform=val_transforms,
            orig_keys="image",  # get the previously applied pre_transforms information on the `img` data field,
                              # then invert `pred` based on this information. we can use same info
                              # for multiple fields, also support different orig_keys for different fields
            meta_keys=["pred_meta_dict","label_meta_dict"],  # key field to save inverted meta data, every item maps to `keys`
            orig_meta_keys="image_meta_dict",  # get the meta data from `img_meta_dict` field when inverting,
                                             # for example, may need the `affine` to invert `Spacingd` transform,
                                             # multiple fields can use the same meta data to invert
            meta_key_postfix="meta_dict",  # if `meta_keys=None`, use "{keys}_{meta_key_postfix}" as the meta key,
                                           # if `orig_meta_keys=None`, use "{orig_keys}_{meta_key_postfix}",
                                           # otherwise, no need this arg during inverting
            nearest_interp=True,  # change to use "nearest" mode in interpolation when inverting
            to_tensor=True,  # convert to PyTorch Tensor after inverting
        ),
        #SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir="./out", output_postfix="seg", resample=False),
    ])

In [ ]:
ds = CacheDataset(data=val_files, transform=val_transforms,cache_rate=1.0, num_workers=2)
test_loader=DataLoader(ds, batch_size=1, shuffle=False,num_workers=2, collate_fn=list_data_collate)
sample=next(iter(test_loader))

In [ ]:
sample['image'].shape,sample['label'].shape

In [ ]:
post_data=[]
for d in test_loader:
    images = d["image"].to(device)
    # define sliding window size and batch size for windows inference
    infer_outputs = sliding_window_inference(inputs=images, roi_size=(32 , 32, 32), sw_batch_size=2, predictor=model)
    infer_outputs = decollate_batch(infer_outputs)
    for (infer_output, infer_output_data) in zip(infer_outputs, decollate_batch(d)):
      infer_output_data["pred"] = infer_output
      post_data.append(post_transforms(infer_output_data))
      break
    break

In [ ]:
out=post_data[0]
out['image'].shape,out['label'].shape,out['pred'].shape

In [ ]:
raw_val=raw_val_data[0]
raw_val[0].shape,raw_val[1].shape#raw label and segment

In [ ]:
fig,ax=plt.subplots(33,3,figsize=(10,60))

for i in range(33):
  ax[i,0].imshow(raw_val[1][:,:,i],cmap='gray')#raw image
  ax[i,1].imshow(out['label'][0,:,:,i],cmap='gray')#inverted image
  ax[i,2].imshow(torch.argmax(out['pred'],0)[:,:,i],cmap='gray')#predicted image
plt.subplots_adjust(wspace=0, hspace=-.5)
